In [1]:
import os
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.nddata import block_reduce
from acstools.findsat_mrt import TrailFinder
import matplotlib.pyplot as plt

def find_latest_dictionary_csv(folder_path):
    csv_files = [f for f in os.listdir(folder_path) if f.startswith('dictionary') and f.endswith('.csv')]
    if not csv_files:
        print("No dictionary CSV file found.")
        return None
    latest_csv = max(csv_files)
    return latest_csv

# Define the directory containing the FITS files
fits_dir = './Data/fits2_ext/'
# Define the directory to save filtered images
filtered_dir = './Data/fits_filtered2_test/'
os.makedirs(filtered_dir, exist_ok=True)

latest_csv = find_latest_dictionary_csv(fits_dir)

if latest_csv:
    csv_path = os.path.join(fits_dir, latest_csv)
    dictionary_df = pd.read_csv(csv_path)
    display(dictionary_df)
    
    # Get a list of FITS files in the directory
    fits_files = [f for f in os.listdir(fits_dir) if f.endswith('.fit')]
    
    # Loop over each FITS file
    for fits_file in fits_files:
        # Construct the full path to the FITS file
        fits_path = os.path.join(fits_dir, fits_file)
        
        image = fits.getdata(fits_path, ext=0)
        image = image - np.nanmedian(image)
        
        # Reduce the image size
        reduced_image = block_reduce(image, 1, func=np.nansum)
        
        # Apply the TrailFinder filter
        s = TrailFinder(reduced_image, processes=2, threshold=2, min_length=200, min_persistence=0.4, max_width=30)
        # ^sprawdzic co to jest, po co to? byc moze da sie to zredukowac bo chyba szuka kresek niepotrzebnie? 
        # albo wgl nie potrzebna linia bo PNGowanie jest niżej za pomocą matplotliba?
        # tu chyba sie nie dalo samego filtra uzyc i trzeba bylo zrobic to w ten sposob
        s.plot_image(overlay_mask=False)
        
        plt.axis('off')
        plt.title('')
        
        # Save the filtered image without white margins
        filtered_image_path = os.path.join(filtered_dir, f'{os.path.splitext(fits_file)[0]}.png')
        plt.savefig(filtered_image_path, bbox_inches='tight')
        plt.close()
        print(f"Filtered image saved: {filtered_image_path}")
        
        # Update the filename in the CSV file
        old_filename = os.path.splitext(fits_file)[0] + '.fit'
        new_filename = os.path.splitext(fits_file)[0] + '.png'
        dictionary_df['output'] = dictionary_df['output'].replace(old_filename, new_filename)

    # Save the updated CSV file
    updated_csv_path = os.path.join(filtered_dir, latest_csv)
    display(dictionary_df)
    dictionary_df.to_csv(updated_csv_path, index=False)
    print(f"Updated CSV file saved: {updated_csv_path}")

,input,output,label
0,tic1.fit,tic1.fit,1
1,tic2.fit,tic2.fit,1
2,tic3.fit,tic3.fit,1
3,A012_ PSN_J12355230plus2755559_R_600s_ZWO ASI1...,tic4.fit,0
4,MPC_Cslash2022-QE78_B_180s_ZWO ASI1600MM-Cool_...,tic5.fit,0
...,...,...,...
138,A012_ PSN_J12355230plus2755559_R_600s_ZWO ASI1...,tic139.fit,0
139,A012_ PSN_J12355230plus2755559_R_600s_ZWO ASI1...,tic140.fit,0
140,A012_ PSN_J12355230plus2755559_R_600s_ZWO ASI1...,tic141.fit,0
141,CD-31_3271_ingress_1_R_20s_ZWO ASI1600MM-Cool_...,tic142.fit,0


Filtered image saved: ./Data/fits_filtered2_test/tic1.png
Filtered image saved: ./Data/fits_filtered2_test/tic10.png
Filtered image saved: ./Data/fits_filtered2_test/tic100.png
Filtered image saved: ./Data/fits_filtered2_test/tic101.png
Filtered image saved: ./Data/fits_filtered2_test/tic102.png
Filtered image saved: ./Data/fits_filtered2_test/tic103.png
Filtered image saved: ./Data/fits_filtered2_test/tic104.png
Filtered image saved: ./Data/fits_filtered2_test/tic105.png
Filtered image saved: ./Data/fits_filtered2_test/tic106.png
Filtered image saved: ./Data/fits_filtered2_test/tic107.png
Filtered image saved: ./Data/fits_filtered2_test/tic108.png
Filtered image saved: ./Data/fits_filtered2_test/tic109.png
Filtered image saved: ./Data/fits_filtered2_test/tic11.png
Filtered image saved: ./Data/fits_filtered2_test/tic110.png
Filtered image saved: ./Data/fits_filtered2_test/tic111.png
Filtered image saved: ./Data/fits_filtered2_test/tic112.png
Filtered image saved: ./Data/fits_filtered2_

,input,output,label
0,tic1.fit,tic1.png,1
1,tic2.fit,tic2.png,1
2,tic3.fit,tic3.png,1
3,A012_ PSN_J12355230plus2755559_R_600s_ZWO ASI1...,tic4.png,0
4,MPC_Cslash2022-QE78_B_180s_ZWO ASI1600MM-Cool_...,tic5.png,0
...,...,...,...
138,A012_ PSN_J12355230plus2755559_R_600s_ZWO ASI1...,tic139.png,0
139,A012_ PSN_J12355230plus2755559_R_600s_ZWO ASI1...,tic140.png,0
140,A012_ PSN_J12355230plus2755559_R_600s_ZWO ASI1...,tic141.png,0
141,CD-31_3271_ingress_1_R_20s_ZWO ASI1600MM-Cool_...,tic142.png,0


Updated CSV file saved: ./Data/fits_filtered2_test/dictionary_0.csv
